In [9]:
import json
import numpy as np

In [2]:
datafile = "reviews_Baby_5.json/datawithcomma"
jsonfile = open(datafile)
data = json.load(jsonfile)
userlist ={x['reviewerID']:[] for x in data}
# uniqreviewer = list(set(reviewerIds))

userlist1 = list(map(lambda item:userlist[item["reviewerID"]].append((item["asin"],item["unixReviewTime"])),data))


In [3]:
removelist = [k for k in userlist if len(userlist[k]) < 10]

i=0
itemset=set()
print(len(userlist))
for key in removelist:
    del userlist[key]
for k in userlist:    
    userlist[k].sort(key = lambda x:x[1])
#     print(userlist[k][:][0])
    itemset = itemset.union(set([a for (a,b) in userlist[k]]))
    
print(len(userlist))



19445
4382


In [15]:

encodedict = {val:idx for idx,val in enumerate(list(itemset),start=1)}

# print(encodedict)

# featurevector = [[encodedict[x] for (x,y) in useritems] for useritems in userlist.values()]
featurevector=[]
maxlen = 0
for userseq in userlist.values():
#     featurevector.append([])
    (featurevector.append([encodedict[x] for (x,y) in userseq]))
    if len(featurevector[-1]) > maxlen:
        maxlen = len(featurevector[-1])
        


In [24]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

# The Embedding layer takes at least two arguments:
# the number of possible words in the vocabulary, here 1000 (1 + maximum word index),
# and the dimensionality of the embeddings, here 32.
embedding_dim=32




In [18]:

train_data = keras.preprocessing.sequence.pad_sequences(featurevector,
                                                        value=0,
                                                        padding='post',
                                                        maxlen=maxlen)


In [22]:
print(train_data[0])


[ 608 3290 4511 2484 1339 4464 4038 5214 4919 1967 6720    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0]


In [25]:


model = keras.Sequential([
  layers.Embedding(len(itemset), embedding_dim, input_length=maxlen)
  
])

model.summary()


Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 125, 32)           220256    
Total params: 220,256
Trainable params: 220,256
Non-trainable params: 0
_________________________________________________________________


In [4]:
import pandas as pd
import numpy as np 

filename = 'ratings_Baby.csv'

df = pd.read_csv(filename,names=['userid','itemid','rating','timestamp'])
userset = np.unique(df.iloc[:,0].values)
itemset = np.unique(df.iloc[:,1].values)
userdict = {userset[i]:i for i in range(len(userset))}
itemdict = {itemset[i]:i for i in range(len(itemset))}
df['user_id']= df.apply(lambda row : userdict[row.userid],axis=1)
df['item_id']= df.apply(lambda row : itemdict[row.itemid],axis=1)
df = df.drop(columns=['userid','itemid'])
df = df.sort_values(['user_id','timestamp'],ascending=[True,True])

print(df.head())

        rating   timestamp  user_id  item_id
565289     5.0  1402272000        0    28438
661829     5.0  1402272000        0    35116
706586     5.0  1404432000        1    38846
114426     5.0  1363996800        2     2778
857429     3.0  1395619200        3    53525


In [5]:
print(df.iloc[0:40,:])

        rating   timestamp  user_id  item_id
565289     5.0  1402272000        0    28438
661829     5.0  1402272000        0    35116
706586     5.0  1404432000        1    38846
114426     5.0  1363996800        2     2778
857429     3.0  1395619200        3    53525
830052     5.0  1404259200        3    50755
558474     4.0  1381363200        4    27981
438858     4.0  1402617600        4    20593
781486     5.0  1402617600        4    44982
869648     5.0  1402617600        4    55421
900485     5.0  1402617600        4    60787
422794     3.0  1404259200        4    19679
663457     5.0  1404259200        4    35230
907036     3.0  1404259200        4    62145
853569     3.0  1405209600        4    53078
763901     5.0  1363651200        5    43161
76675      4.0  1387411200        5     1489
434713     4.0  1387411200        5    20347
547713     5.0  1354233600        6    27202
800569     5.0  1370304000        7    47210
490426     5.0  1403654400        8    23446
438191    